In [1]:
import os
import csv
import jsonlines
from collections import Counter

In [2]:
import re
import spacy
import random
import json

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [7]:
nlp = spacy.load("en_core_web_lg")

In [29]:
project_folder='./../Master_Thesis' 
shuffle_num=1

data_path = os.path.join(project_folder, 'Data/thesis_data_shuffle-' + str(shuffle_num) + '_.json')
with open(data_path) as reader:
    data_ = json.load(reader)

In [32]:
data_['metadata']['enrichments'] = {
    'tokenization': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.text'        
    },
    'part-of-speech': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.tag_'
    },
    'named-entities': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.ent_type_'
    }
}

In [33]:
from pprint import pprint
pprint(data_['metadata'])

{'article_nums': {'test': 32, 'train': 251, 'valid': 32},
 'enrichments': {'named-entities': {'key': 'token.ent_type_',
                                    'tool': 'spaCy',
                                    'version': '2.0.12'},
                 'part-of-speech': {'key': 'token.tag_',
                                    'tool': 'spaCy',
                                    'version': '2.0.12'},
                 'tokenization': {'key': 'token.text',
                                  'tool': 'spaCy',
                                  'version': '2.0.12'}},
 'label_nums': {'test': {'0': 320, '1': 116, '2': 5},
                'train': {'0': 2472, '1': 1053, '2': 57},
                'valid': {'0': 268, '1': 130, '2': 1}},
 'raw_data_path': '/home/aorus/workspaces/simge/corpus/Sentence/Newindianexpress/20181001-newindianexpress_sentence_classification_adjudicated_20181218.json',
 'sentence_nums': {'test': 441, 'train': 3582, 'valid': 399},
 'split_ratio': {'test': 10, 'train': 80, 'valid'

In [34]:
for key, articles in data_['data'].items():
    for article in articles:
        for sent_ind, sentence in enumerate(article['sentences']):
            
            doc = nlp(sentence['sentence'])

            article['sentences'][sent_ind]['tokens'] = [token.text for token in doc] 
            article['sentences'][sent_ind]['pos'] = [token.tag_ for token in doc] 
            article['sentences'][sent_ind]['ner'] = [token.ent_type_ if token.ent_iob_ != 'O' else 'O' 
                                                     for token in doc]
        

In [35]:
with open('Data/thesis_data_shuffle-' + str(shuffle_num) + '_enriched_.json', 'w') as outfile:
    json.dump(data_, outfile)

In [39]:
from wikipedia2vec import Wikipedia2Vec

In [ ]:
MODEL_FILE=''

In [ ]:
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

## CHECK 

In [44]:
import pickle
opt_results_path = 'Scikit-Experiments/Results/results_070919_tfidf_only_mindf_001_maxdf_0_12_numberstoplwords.pickle'

In [45]:
with open(opt_results_path, 'rb') as file_:
    opt_results = pickle.load(file_)

In [46]:
opt_results.keys()

dict_keys(['DecisionTree'])

## Heidel Time